This notebook compares the GA score vs the one of the MILP to validate that the GA can achieve the global optimum.

The validation is done without the network metrics of system accessibility and inter-station proximity. 

Two experiments are done:

1. GA vs MILP score across multiple weights
2. GA vs MILP score across multiple numbers of stations 

---
Author: Jordi Grau Escolano


# 1. Load data

In [5]:
import sys
from pathlib import Path
import pandas as pd
import ast
import numpy as np
from scipy import stats
project_root = Path().resolve().parents[0]
sys.path.insert(0, str(project_root))

from paths import *

ROOT = '..'

# Load data
file_ga_w = pd.read_csv(f'{ROOT}/{PR_EXP}/GA_MILP_weights/GA_results.csv')
file_milp_w = pd.read_csv(f'{ROOT}/{PR_EXP}/GA_MILP_weights/MILP_results.csv')

file_ga_s = pd.read_csv(f'{ROOT}/{PR_EXP}/GA_MILP_stations/GA_results.csv')
file_milp_s = pd.read_csv(f'{ROOT}/{PR_EXP}/GA_MILP_stations/MILP_results.csv')


def prepare_data(file_ga, file_lp):

    df_ga = pd.DataFrame(file_ga)
    df_milp = pd.DataFrame(file_lp)
    df_ga.rename(columns={
        'minutes_to_complete': 'time', 
        'best_score': 'score'}, inplace=True)
    df_milp.rename(columns={
        'minutes_to_complete': 'time',
        'best_score': 'score'}, inplace=True)

    # # Parse weights
    # def parse_weights(weights_str):
    #     weights = ast.literal_eval(weights_str)
    #     return {k.strip(): float(v) for k, v in [item.split(':') for item in weights]}
    
    # Parse weights
    # df_ga['weights'] = df_ga['weights'].map(parse_weights)
    # df_milp['weights'] = df_milp['weights'].map(parse_weights)

    # Weights are too long to display, so we'll create a weight_to_idx dictionary
    weight_to_idx = {weights: experiment_idx for idx, (weights, experiment_idx) in enumerate(df_ga[['weights', 'experiment_idx']].values)}

    # Map weights to indices
    df_ga['weights'] = df_ga['weights'].map(weight_to_idx)
    df_milp['weights'] = df_milp['weights'].map(weight_to_idx)

    # Merge data
    df = pd.merge(df_ga, df_milp, on=['experiment_idx', 'N_stations', 'weights'], suffixes=('_ga', '_lp'), how='outer')

    # Calculate GA accuracy when compared with linear programming
    df['accuracy_ga'] = df['score_ga'] / df['score_lp']
    df['accuracy_bool'] = df['accuracy_ga'] > 0.95
    df_table = df[['experiment_idx', 'N_stations', 'weights', 'score_ga', 'score_lp', 'accuracy_ga', 'accuracy_bool', 'time_ga', 'time_lp']]
    df_table.sort_values('weights')

    # Create pivot table
    pivot_df = pd.pivot_table(
        df_table.round(3),
        index=['experiment_idx', 'weights'],
        columns='N_stations',
        values=['score_ga', 'score_lp', 'accuracy_ga', 'accuracy_bool', 'time_ga', 'time_lp'],
        aggfunc='first')

    # Swap the levels of the columns
    pivot_df = pivot_df.swaplevel(0, 1, axis=1).sort_index(axis=1)

    # Reorder the metrics within each N_stations group
    metric_order = ['score_lp', 'score_ga', 'accuracy_ga', 'accuracy_bool', 'time_lp', 'time_ga']
    ordered_columns = []
    for n_stations in (pivot_df.columns.levels[0]):
        for metric in metric_order:
            ordered_columns.append((n_stations, metric))
    pivot_df = pivot_df.reindex(columns=ordered_columns)

    # Fillna
    pivot_df = pivot_df.fillna(0)

    return pivot_df, weight_to_idx


# Prepare data for weights and stations
df_weights, weight_to_idx_w = prepare_data(file_ga_w, file_milp_w)
df_stations, weight_to_idx_s = prepare_data(file_ga_s, file_milp_s)

# Weights have an index to ease the visualization of the results
for key, value in weight_to_idx_w.items():
    if value <= 2:
        print(value, key)

df_weights.head()

2 ['10-19: 0.03489456438290895', '20-29: 0.016417594502611852', '30-39: 0.02739160248198541', '40-49: 0.02879483074285447', '50-59: 0.009736149035489139', '60-69: 0.051067313533762274', '70+: 0.04082567914897312', 'altitude: 0.014291755135009458', 'bike_lane_kms: 0.01878993691399383', 'bus_lines: 0.002382088243706499', 'cars: 0.047129902387407524', 'education_college: 0.0018112182675607538', 'education_primary: 0.006427664613757233', 'education_secondary: 0.02608060585864848', 'f: 0.047893240822790756', 'has_bike_lane: 0.043648998762104485', 'household_avg_m2: 0.04948272243559799', 'income_2022_pers: 0.023182510073711356', 'm: 0.01362975245090035', 'metro_lines: 0.02047134870711412', 'motos: 0.03149095479181811', 'n_civic: 0.00029084587198059247', 'n_culture: 0.028583405964349422', 'n_economic_retail: 0.0519786134913751', 'n_education: 0.037229659555487585', 'n_green: 0.010466209676175906', 'n_health_care: 0.014796615249291043', 'n_industrial: 0.04067356745269744', 'n_recreation: 0.042

N_stations                   60                                             \
                       score_lp score_ga accuracy_ga accuracy_bool time_lp   
experiment_idx weights                                                       
1              1         53.424   53.107       0.994          True   5.858   
2              2         53.349   53.169       0.997          True   5.843   
3              3         51.700   51.200       0.990          True   5.805   
4              4         53.662   53.585       0.999          True   5.837   
5              5         53.625   53.355       0.995          True   5.804   

N_stations                      
                       time_ga  
experiment_idx weights          
1              1        33.803  
2              2        28.252  
3              3        31.718  
4              4        36.653  
5              5        33.019

# 2. GA vs MILP score comparison  

In [33]:
def generate_summary_statistics(pivot_df):
    """Generate summary statistics for each N_stations"""
    summaries = {}

    # Drop columns with all zeros
    columns = pivot_df.loc[:, (pivot_df != 0).any(axis=0)].columns
    n_stations = [col[0] for col in columns]
    n_stations_list = sorted(set(n_stations))

    # For each N_stations group
    for n in n_stations_list: 
        # Get the data for this N_stations
        station_data = pivot_df[n].dropna()
        round_to = 2

        # Calculate statistics
        # Use only valid GA and MILP solutions
        valid_solutions = station_data[
            (station_data['accuracy_ga'] > 0) &
            (station_data['score_lp'] > 0)]
        
        # If there are valid solutions, compute averages and std.
        if len(valid_solutions) > 0:
            avg_ga_score = valid_solutions['score_ga'].mean()
            std_ga_score = valid_solutions['score_ga'].std()
            avg_acc = valid_solutions['accuracy_ga'].mean()
            std_acc = valid_solutions['accuracy_ga'].std()
            avg_ga_time = valid_solutions['time_ga'].mean()
            std_ga_time = valid_solutions['time_ga'].std()
            
            avg_lp_score = valid_solutions['score_lp'].mean()
            std_lp_score = valid_solutions['score_lp'].std()
            avg_lp_time = valid_solutions['time_lp'].mean()
            std_lp_time = valid_solutions['time_lp'].std()

        else:
            avg_ga_score = std_ga_score = avg_acc = std_acc = avg_ga_time = std_ga_time = 0
        
        statistics = {
            f"N={n}": {
                "Number of experiments": len(valid_solutions),
                "Avg LP score (Std)": f"{avg_lp_score:.{round_to}f} ({std_lp_score:.{round_to}f})",
                "Avg GA score (Std)": f"{avg_ga_score:.{round_to}f} ({std_ga_score:.{round_to}f})",
                "Accuracy": f"{avg_acc:.{round_to}f} ({std_acc:.{round_to}f})",
                "Avg LP time (Std)": f"{avg_lp_time:.{round_to}f} ({std_lp_time:.{round_to}f})",
                "Avg GA time (Std)": f"{avg_ga_time:.{round_to}f} ({std_ga_time:.{round_to}f})",
                "Good solutions": valid_solutions['accuracy_bool'].sum(),
                "Bad solutions": len(valid_solutions) - valid_solutions['accuracy_bool'].sum()
            }
        }
        summaries.update(statistics)
    
    # Convert to DataFrame for nice display
    summary_df = pd.DataFrame(summaries).T

    # Perform power analysis and statistical tests between LP and GA scores for each N_stations
    # Threshold for statistical significance in percentage
    threshold_percent = 0.5
    for n in n_stations_list:
        # Get the data for this N_stations
        station_data = pivot_df[n].dropna()
        
        # Only compare where GA produced valid solutions
        valid_solutions = station_data[
            (station_data['score_ga'] != 0) &
            (station_data['score_lp'] != 0)]
        
        if len(valid_solutions) > 0:
            lp_scores = valid_solutions['score_lp']
            ga_scores = valid_solutions['score_ga']
            
            # Compute the percentage difference:
            # If GA is 99% of LP, then (LP - GA) / LP * 100 = 1.
            differences_percent = (lp_scores - ga_scores) / lp_scores * 100

            # Test normality of these differences.
            _, p_normality = stats.shapiro(differences_percent)
            is_normal = p_normality > 0.05
            print(f"N={n}: Shapiro p={p_normality:.3f} ({'normal' if is_normal else 'not normal'})")
            
            if is_normal:
                # When normal, perform a one-sample t-test:
                # H0: mean(differences) >= threshold_percent 
                # H1: mean(differences) < threshold_percent
                t_stat, p_two_tailed = stats.ttest_1samp(differences_percent, popmean=threshold_percent)
                # Convert the two-tailed p-value into a one-tailed p-value.
                if t_stat < 0:
                    p_one_tailed = p_two_tailed / 2
                else:
                    p_one_tailed = 1 - (p_two_tailed / 2)
                print("Test: One-sample t-test on percentage differences")
                print(f"t-statistic = {t_stat:.4f}, one-tailed p-value = {p_one_tailed:.4f}")
            else:
                # When non-normal, use the Wilcoxon signed-rank test.
                # Adjust the differences by subtracting the threshold so that H0: median(adjusted_diff)=0.
                adjusted_diff = differences_percent - threshold_percent
                wilcoxon_stat, p_two_tailed = stats.wilcoxon(adjusted_diff)
                if np.median(adjusted_diff) < 0:
                    p_one_tailed = p_two_tailed / 2
                else:
                    p_one_tailed = 1 - (p_two_tailed / 2)
                print("Test: Wilcoxon signed-rank test on adjusted percentage differences")
                print(f"Wilcoxon statistic = {wilcoxon_stat:.1f}, one-tailed p-value = {p_one_tailed:.4f}")

            
            # ------------------------
            # Perform a power analysis for this experiment
            # ------------------------
            observed_mean = differences_percent.mean()

            # Only do the power analysis if the observed mean difference is less than the threshold.
            threshold_percent = 0.5
            if observed_mean < threshold_percent:
                
                # Compute the effect size as the difference between the threshold and the observed mean difference.
                # Also compute the variability.
                delta_effect = threshold_percent - observed_mean
                sigma = differences_percent.std()
                print(f"threshold_percent = {threshold_percent:.2f}, observed_mean = {observed_mean:.2f}, delta_effect = {delta_effect:.2f}, sigma = {sigma:.2f}")

                # Set the significance level for the test. We want less than 0.1% chance of Type I error.
                alpha = 0.001
                # Set the desired power (Error type II is 1 - desired_power). We want a 99% chance of detecting the effect if it exists.
                desired_power = 0.9999
                print(f"desired_power = {desired_power:.4f}, alpha = {alpha:.4f}")

                # Calculate the critical z-value for the chosen significance level. As it is one-sided -> 1 - alpha.
                z_alpha = stats.norm.ppf(1 - alpha)

                # Calculate the critical z-value corresponding to the desired power.
                # This is the z-value such that the area to the left equals the desired power.
                z_beta = stats.norm.ppf(desired_power)

                # Compute the required sample size (n) using the standard formula for power analysis in a paired design.
                # This formula approximates how many paired observations (valid experiments) are needed.
                n_estimate = ((z_alpha + z_beta) * sigma / delta_effect) ** 2

                # Since the sample size must be an integer, round the computed value up to the nearest whole number.
                required_n = int(np.ceil(n_estimate))

                # Print the estimated required sample size based on the desired power and significance level.
                print(f"Estimated required sample size (power={desired_power}): {required_n}")
            else:
                # If the observed mean difference is not below the threshold, the data do not support the alternative hypothesis.
                # In this case, performing a power analysis for detecting a difference below the threshold is not applicable.
                print("Observed mean difference is not below the threshold; power analysis not applicable.")

            # Print the final one-tailed p-value from the hypothesis test and the number of valid experiments used.
            # This gives an idea whether the test is significant relative to the significance level (alpha)
            # and how many paired observations (experiments) you had.
            print(f"(p = {p_one_tailed:.4f}), {'<' if p_one_tailed < alpha else '>'} than significance level, n = {len(valid_solutions)}\n")
            
    cols_order = ['Avg LP score (Std)', 'Avg GA score (Std)', 'Avg LP time (Std)', 'Avg GA time (Std)', 'Accuracy', 'Bad solutions', 'Good solutions', 'Number of experiments']
    return summary_df[cols_order]

At least n paired samples are needed to be 99.99% confident of detecting that the GA algorithm's performance is better than the threshold (0.5%), with only a 0.01% chance of falsely claiming it's better when it's not.

In [31]:
# Statistics for 60 stations
summary_stats = generate_summary_statistics(df_weights[[60]])
print("\nSummary Statistics:")
summary_stats

# This is a “good” statistical conclusion in the sense that there is strong evidence
# against the null hypothesis, which states that the GA score is not within
# the 5% margin of the LP score

N=60: Shapiro p=0.017 (not normal)
Test: Wilcoxon signed-rank test on adjusted percentage differences
Wilcoxon statistic = 965.0, one-tailed p-value = 0.0000
threshold_percent = 0.50, observed_mean = 0.35, delta_effect = 0.15, sigma = 0.19
desired_power = 0.9999, alpha = 0.0010
Estimated required sample size (power=0.9999): 76
(p = 0.0000), < than significance level, n = 118


Summary Statistics:


,Avg LP score (Std),Avg GA score (Std),Avg LP time (Std),Avg GA time (Std),Accuracy,Bad solutions,Good solutions,Number of experiments
N=60,53.48 (0.51),53.29 (0.54),5.92 (0.07),33.60 (10.25),1.00 (0.00),0,118,118


In [34]:
# Statistics for 60 stations
summary_stats = generate_summary_statistics(df_stations) 
# 24 experiments leads to a power analysis of 33[30 and 120 stations] and 46 [60 and 90 stations] experiments
print("\nSummary Statistics:")
summary_stats

N=30: Shapiro p=0.000 (not normal)
Test: Wilcoxon signed-rank test on adjusted percentage differences
Wilcoxon statistic = 24.0, one-tailed p-value = 0.0002
threshold_percent = 0.50, observed_mean = 0.23, delta_effect = 0.27, sigma = 0.23
desired_power = 0.9999, alpha = 0.0010
Estimated required sample size (power=0.9999): 33
(p = 0.0002), < than significance level, n = 24

N=60: Shapiro p=0.066 (normal)
Test: One-sample t-test on percentage differences
t-statistic = -5.0070, one-tailed p-value = 0.0000
threshold_percent = 0.50, observed_mean = 0.33, delta_effect = 0.17, sigma = 0.17
desired_power = 0.9999, alpha = 0.0010
Estimated required sample size (power=0.9999): 45
(p = 0.0000), < than significance level, n = 24

N=90: Shapiro p=0.858 (normal)
Test: One-sample t-test on percentage differences
t-statistic = -4.9317, one-tailed p-value = 0.0000
threshold_percent = 0.50, observed_mean = 0.34, delta_effect = 0.16, sigma = 0.16
desired_power = 0.9999, alpha = 0.0010
Estimated required

,Avg LP score (Std),Avg GA score (Std),Avg LP time (Std),Avg GA time (Std),Accuracy,Bad solutions,Good solutions,Number of experiments
N=30,27.73 (0.10),27.67 (0.13),5.92 (0.08),10.51 (2.76),1.00 (0.00),0,24,24
N=60,53.44 (0.43),53.26 (0.46),5.93 (0.11),21.45 (4.69),1.00 (0.00),0,24,24
N=90,77.77 (0.77),77.50 (0.76),5.94 (0.06),38.01 (11.86),1.00 (0.00),0,24,24
N=120,100.89 (1.03),100.60 (1.03),5.96 (0.10),65.66 (20.23),1.00 (0.00),0,24,24
